In [ ]:
!pip install tslearn

In [1]:
import sys
sys.path.append("../src")
sys.path.append("../src/models")
import sklearn
import tslearn
import argparse
from train import prepare_dataset
from experiments import experiments
import tqdm
import numpy as np
from tslearn.svm import TimeSeriesSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from utils.classmetric import confusion_matrix_to_accuraccies
from sklearn.metrics import confusion_matrix

from tslearn.utils import to_time_series_dataset
from time import time
from sklearn.model_selection import RandomizedSearchCV
import scipy
from sklearn.ensemble import RandomForestClassifier

from utils.data2numpy import get_data

from fastai_timeseries import *

def flatten(x):
    return x.reshape(x.shape[0], -1)

In [ ]:
args.dataroot

In [2]:
N_per_class = 500

#tum_ds = get_data("isprs_rf_tum_23classes", dataroot='/home/glennmoncrieff/crop-type-mapping',N_per_class=N_per_class, N_largest=None, do_add_spectral_indices=True)
#gaf_ds = get_data("isprs_rf_gaf_23classes", N_per_class=N_per_class, N_largest=None, do_add_spectral_indices=True)

In [3]:

tum12_ds = get_data("isprs_rf_tum_12classes", N_per_class=N_per_class, N_largest=None, do_add_spectral_indices=True)
#gaf12_ds = get_data("isprs_rf_gaf_12classes", dataroot=os.getcwd(),N_per_class=N_per_class, N_largest=None, do_add_spectral_indices=True)

  0%|          | 37/10242 [00:00<00:28, 360.41it/s]

Initializing BavarianCropsDataset test partition in holl
read 12 classes
no cached dataset found. iterating through csv folders in /home/glennmoncrieff/crop-type-mapping/notebooks/data/BavarianCrops/csv/holl


  0%|          | 17/3664 [00:00<00:21, 166.65it/s]

loaded 9583 samples
Dataset /home/glennmoncrieff/crop-type-mapping/notebooks/data/BavarianCrops. region holl. partition test. X:9583x(144, 13), y:(9583,) with 12 classes
Initializing BavarianCropsDataset test partition in nowa
read 12 classes
no cached dataset found. iterating through csv folders in /home/glennmoncrieff/crop-type-mapping/notebooks/data/BavarianCrops/csv/nowa


  1%|          | 47/4492 [00:00<00:09, 465.93it/s]

...

  0%|          | 53/26064 [00:00<00:49, 527.48it/s]

loaded 8425 samples
Dataset /home/glennmoncrieff/crop-type-mapping/notebooks/data/BavarianCrops. region nowa. partition trainvalid. X:8425x(289, 13), y:(8425,) with 12 classes
Initializing BavarianCropsDataset trainvalid partition in krum
read 12 classes
no cached dataset found. iterating through csv folders in /home/glennmoncrieff/crop-type-mapping/notebooks/data/BavarianCrops/csv/krum


100%|██████████| 26064/26064 [00:56<00:00, 461.18it/s]
3it [00:00, 27.76it/s]

loaded 25083 samples
Dataset /home/glennmoncrieff/crop-type-mapping/notebooks/data/BavarianCrops. region krum. partition trainvalid. X:25083x(71, 13), y:(25083,) with 12 classes
setting random seed to 0


228it [00:07, 29.29it/s]
68it [00:02, 30.41it/s]


# Support Vector Machine

## TUM dataset

In [ ]:
!cat /data/isprs/sklearn/svm_tum.txt
#svm_tum_params = {'C': 34.94762791972138, 'gamma': 0.09632187442535493, 'kernel': 'rbf'}
#svm_tum_params ={'C': 219.5746177088989, 'gamma': 0.030512293944411368, 'kernel': 'rbf'}
svm_tum_params ={'C': 219.5746177088989, 'gamma': 0.030512293944411368, 'kernel': 'rbf'}
!cat /data/isprs/sklearn/svm_gaf.txt
svm_gaf_params = {'C': 83.60975452306279, 'gamma': 0.03084148398718425, 'kernel': 'rbf'}

In [ ]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = tum_ds

clf = TimeSeriesSVC(n_jobs=8,**svm_tum_params)
clf.fit(X, y)
y_pred = clf.predict(Xtest)
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

In [ ]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = gaf_ds

clf = sklearn.svm.SVC(**svm_tum_params)
#clf = TimeSerxiesSVC(n_jobs=8,**svm_tum_params)
clf.fit(feat_ext(X), y)
y_pred = clf.predict(feat_ext(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

In [ ]:
!pip install tsfresh
from tsfresh import extract_features
extracted_features = extract_features(X[0,:,0])

In [ ]:
def feat_ext(X):
    f1 = X.mean(1)
    f2 = X.std(1)
    f3 = np.median(X,1)
    return np.hstack([f1,f2,f2])

## GAF Dataset

In [ ]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = gaf_ds

clf = TimeSeriesSVC(n_jobs=8,**svm_gaf_params)
clf.fit(X, y)
y_pred = clf.predict(Xtest)
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

# Random Forest and Rocket

### RF setup

In [4]:
from sklearn.ensemble import RandomForestClassifier
params = {'n_estimators': 1400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': False}


In [5]:
!cat /data/isprs/sklearn/random_forest_tum.txt
rf_tum_params = {'bootstrap': False, 'max_depth': 70, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 2000}
#rf_tum_params = {'bootstrap': False, 'max_depth': 50, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 2000}
!cat /data/isprs/sklearn/random_forest_gaf.txt
rf_gaf_params = {'bootstrap': False, 'max_depth': 60, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 1000}
#rf_gaf_params = {'bootstrap': False, 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1200}

cat: /data/isprs/sklearn/random_forest_tum.txt: No such file or directory
cat: /data/isprs/sklearn/random_forest_gaf.txt: No such file or directory


### Rocket setup

In [11]:
from fastai_timeseries import *
print('pytorch:', torch.__version__)
print('fastai :', fastai.__version__)

pytorch: 1.3.0
fastai : 1.0.59


In [12]:
import torch
import torch.nn as nn

class ROCKET(nn.Module):
    def __init__(self, c_in, seq_len, n_kernels=10000, kss=[7, 9, 11]):
        
        '''
        ROCKET is a GPU Pytorch implementation of the ROCKET methods generate_kernels 
        and apply_kernels that can be used  with univariate and multivariate time series.
        Input: is a 3d torch tensor of type torch.float32. When used with univariate TS, 
        make sure you transform the 2d to 3d by adding unsqueeze(1).
        c_in: number of channels or features. For univariate c_in is 1.
        seq_len: sequence length
        '''
        super().__init__()
        kss = [ks for ks in kss if ks < seq_len]
        convs = nn.ModuleList()
        for i in range(n_kernels):
            ks = np.random.choice(kss)
            dilation = 2**np.random.uniform(0, np.log2((seq_len - 1) // (ks - 1)))
            padding = int((ks - 1) * dilation // 2) if np.random.randint(2) == 1 else 0
            weight = torch.randn(1, c_in, ks)
            weight -= weight.mean()
            bias = 2 * (torch.rand(1) - .5)
            layer = nn.Conv1d(c_in, 1, ks, padding=2 * padding, dilation=int(dilation), bias=True)
            layer.weight = torch.nn.Parameter(weight, requires_grad=False)
            layer.bias = torch.nn.Parameter(bias, requires_grad=False)
            convs.append(layer)
        self.convs = convs
        self.n_kernels = n_kernels
        self.kss = kss

    def forward(self, x):
        for i in range(self.n_kernels):
            out = self.convs[i](x)
            _max = out.max(dim=-1).values
            _ppv = torch.gt(out, 0).sum(dim=-1).float() / out.shape[-1]
            cat = torch.cat((_max, _ppv), dim=-1)
            output = cat if i == 0 else torch.cat((output, cat), dim=-1)
        return output

### Rocket eg

In [14]:
X_train, y_train, X_valid, y_valid = get_UCR_data('HandMovementDirection')
X_train.shape

_, features, seq_len = X_train.shape
X_train = (X_train - X_train.mean(axis=(1, 2), keepdims=True)) / (
    X_train.std(axis=(1, 2), keepdims=True) + 1e-8)
X_valid = (X_valid - X_valid.mean(axis=(1, 2), keepdims=True)) / (
    X_valid.std(axis=(1, 2), keepdims=True) + 1e-8)
X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_valid = torch.tensor(X_valid, dtype=torch.float32, device=device)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

n_kernels=100
kss=[7, 9, 11]
model = ROCKET(features, seq_len, n_kernels=n_kernels, kss=kss).to(device)

X_train_tfm = model(X_train)
X_valid_tfm = model(X_valid)

X_train_tfm.shape


ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17), normalize=True)
ridge.fit(X_train_tfm, y_train)

rf_tum = RandomForestClassifier(n_jobs=-1,**rf_tum_params)
rf_tum.fit(flatten(X),y)
y_pred = rf_tum.predict(flatten(Xtest)

(160, 10, 400)

## Raw Dataset 23 classes

In [ ]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = tum_ds

rf_tum = RandomForestClassifier(n_jobs=-1,**rf_tum_params)
rf_tum.fit(flatten(X),y)
y_pred = rf_tum.predict(flatten(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

## Preprocessed Dataset 23 classes

In [ ]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = gaf_ds

rf_tum = RandomForestClassifier(n_jobs=-1,**rf_gaf_params)
rf_tum.fit(flatten(X),y)
y_pred = rf_tum.predict(flatten(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

## raw Dataset 12 classes

In [68]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = tum12_ds

In [69]:
### Rocket + RF

In [70]:
obsx,tsx,bax = Xtest.shape
obs,ts,ba = X.shape

In [71]:
X_r= X.reshape(obs,ba,ts)
Xtest_r= Xtest.reshape(obsx,bax,tsx)

In [72]:
_, features, seq_len = X_r.shape
X_t = torch.tensor(X_r, dtype=torch.float32, device=device)
X_p = torch.tensor(Xtest_r, dtype=torch.float32, device=device)
n_kernels=1000
kss=[5,10,20,40,70]
model = ROCKET(features, seq_len, n_kernels=n_kernels, kss=kss).to(device)
X_tfm = model(X_t)
Xp_tfm = model(X_p)

In [73]:
X_tfm.shape

torch.Size([4987, 2000])

In [ ]:
rf_tum = RandomForestClassifier(n_jobs=-1,**rf_tum_params)
rf_tum.fit(X_tfm,y)
y_pred = rf_tum.predict(Xp_tfm)

In [66]:
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

                  precision    recall  f1-score   support

          fallow       0.33      0.27      0.30       500
       grassland       0.45      0.34      0.39       500
    winter wheat       0.02      0.00      0.01       500
            corn       0.49      0.38      0.43       500
    summer wheat       0.00      0.00      0.00        56
    winter spelt       0.00      0.00      0.00        52
      winter rye       0.00      0.00      0.00       104
   winter barley       0.19      0.03      0.05       500
   summer barley       0.29      0.44      0.35       500
      summer oat       0.21      0.39      0.28       275
winter triticale       0.16      0.37      0.22       499
        rapeseed       0.24      0.42      0.31       257

        accuracy                           0.27      4243
       macro avg       0.20      0.22      0.19      4243
    weighted avg       0.26      0.27      0.24      4243

overall accuracy = 0.27, kappa = 0.18, precision = 0.20, recall=0.22,

In [ ]:
### Only RF

In [6]:
rf_tum = RandomForestClassifier(n_jobs=-1,**rf_tum_params)
rf_tum.fit(flatten(X),y)
y_pred = rf_tum.predict(flatten(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

                  precision    recall  f1-score   support

          fallow       0.55      0.39      0.46       500
       grassland       0.64      0.57      0.60       500
    winter wheat       0.42      0.22      0.29       500
            corn       0.81      0.79      0.80       500
    summer wheat       0.00      0.00      0.00        56
    winter spelt       0.67      0.04      0.07        52
      winter rye       0.23      0.03      0.05       104
   winter barley       0.52      0.26      0.35       500
   summer barley       0.53      0.47      0.50       500
      summer oat       0.25      0.59      0.35       275
winter triticale       0.25      0.54      0.34       499
        rapeseed       0.80      0.72      0.76       257

        accuracy                           0.47      4243
       macro avg       0.47      0.39      0.38      4243
    weighted avg       0.52      0.47      0.46      4243

overall accuracy = 0.47, kappa = 0.40, precision = 0.47, recall=0.39,

## pre Dataset 12 classes

In [ ]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = gaf12_ds

rf_tum = RandomForestClassifier(n_jobs=-1,**rf_tum_params)
rf_tum.fit(flatten(X),y)
y_pred = rf_tum.predict(flatten(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")